In [ ]:
# imports
# type: ignore
from __future__ import annotations
from typing import Any

import aiohttp

In [ ]:
def className(obj: object) -> str:
    return obj.__class__.__name__

def getIndent(depth: int) -> str:
    return '    ' * depth

def printDictTypes(dict_: dict[str, Any], depth: int = 0) -> None:
    for key, value in dict_.items():
        if isinstance(value, dict):
            print(f'{getIndent(depth)}{key}: dict')
            printDictTypes(value, depth + 1)
        else:
            print(f'{getIndent(depth)}{key}: {className(value)}')

async def getFullPlayerStats(username_or_uuid: str) -> tuple[dict[str, Any], dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get(f"https://api.wynncraft.com/v3/player/{username_or_uuid}?fullResult") as resp:
            return (await resp.json(), dict(resp.headers))

async def getGuildStats(guildname_or_prefix: str, is_prefix: bool = False) -> tuple[dict[str, Any], dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get(f"https://api.wynncraft.com/v3/guild/{'prefix/' if is_prefix else ''}{guildname_or_prefix}") as resp:
            return (await resp.json(), dict(resp.headers))

async def getOnlinePlayers() -> tuple[dict[str, Any], dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get("https://api.wynncraft.com/v3/player?identifier=uuid") as resp:
            return (await resp.json(), dict(resp.headers))

# Sandbox

In [ ]:
from kans import config
from kans.db.kans.kans_database import KansDatabase
from kans.logger.kans_logger import KansLogger
from kans.util.api_response_adapter import ApiResponseAdapter
from tests.fixtures_api import FixturesApi

adapter = ApiResponseAdapter()
db = KansDatabase(config, KansLogger(config))
fixturesapi = FixturesApi()

await db.create_all()

In [45]:
from __future__ import annotations

import pandas as pd
import uuid
import mysql.connector

class UuidColumn(pd.Series):
    def to_uuidstring(self) -> pd.Series[str]: return self.apply(
            lambda x: str(uuid.UUID(bytes=bytes(x)))
    )

class PlayerInfoModel(pd.DataFrame):
    @property
    def uuid(self) -> UuidColumn: return UuidColumn(self['uuid'])
    @property
    def latest_username(self) -> pd.Series[str]: return self['latest_username']
    @property
    def datetime(self) -> pd.Series[dt]: return self['datetime']

with mysql.connector.connect(host='localhost', database='wynndata', user='root', password='root') as conn:
    df = PlayerInfoModel(pd.read_sql_query("SELECT * FROM player_info LIMIT 1000", conn))

df.head()



C:\Users\faz\AppData\Local\Temp\ipykernel_16756\4021294994.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = PlayerInfoModel(pd.read_sql_query("SELECT * FROM player_info LIMIT 1000", conn))


,uuid,latest_username,first_join
0,"[0, 172, 62, 139, 254, 107, 71, 24, 170, 237, ...",Sproofless,2018-12-23 18:06:21
1,"[1, 54, 134, 234, 154, 104, 70, 32, 187, 24, 1...",Jazzmyn0,2017-07-11 22:49:58
2,"[1, 134, 175, 81, 199, 96, 79, 237, 162, 109, ...",Moagle,2022-08-11 09:59:13
3,"[2, 32, 81, 144, 102, 149, 67, 137, 135, 40, 1...",MikeyHei8,2024-01-27 04:33:38
4,"[2, 67, 19, 181, 95, 50, 64, 14, 138, 14, 172,...",Nergi,2023-12-25 13:09:33


In [46]:
df.uuid.to_uuidstring()

0      00ac3e8b-fe6b-4718-aaed-b97f885bba7b
1      013686ea-9a68-4620-bb18-84a5cd4def7f
2      0186af51-c760-4fed-a26d-893228c4c628
3      02205190-6695-4389-8728-aa6aac122a38
4      024313b5-5f32-400e-8a0e-ac563677d808
                       ...                 
652    fea8f6a9-4626-4b9e-9c52-0460abe765d8
653    feb5e44d-389e-45b1-99a5-794a6058273c
654    feee8bbc-a528-476a-89f6-0c4835b4bf62
655    ffc35ca6-f9a3-42fa-a236-ddcfca4f3b7a
656    ffe4e872-98ec-4664-ba48-f30e07931751
Name: uuid, Length: 657, dtype: object

In [ ]:
32, 33, 48, 7, 6, 9, 47, 2, 27, 1

# Archive

In [ ]:
# adapter pattern in python
# class Target:
#     """
#     The Target defines the domain-specific interface used by the client code.
#     """

#     def request(self) -> str:
#         return "Target: The default target's behavior."


# class Adaptee:
#     """
#     The Adaptee contains some useful behavior, but its interface is incompatible
#     with the existing client code. The Adaptee needs some adaptation before the
#     client code can use it.
#     """

#     def specific_request(self) -> str:
#         return ".eetpadA eht fo roivaheb laicepS"


# class Adapter(Target, Adaptee):
#     """
#     The Adapter makes the Adaptee's interface compatible with the Target's
#     interface via multiple inheritance.
#     """

#     def request(self) -> str:
#         return f"Adapter: (TRANSLATED) {self.specific_request()[::-1]}"


# def client_code(target: "Target") -> None:
#     """
#     The client code supports all classes that follow the Target interface.
#     """

#     print(target.request(), end="")


# if __name__ == "__main__":
#     print("Client: I can work just fine with the Target objects:")
#     target = Target()
#     client_code(target)
#     print("\n")

#     adaptee = Adaptee()
#     print("Client: The Adaptee class has a weird interface. "
#           "See, I don't understand it:")
#     print(f"Adaptee: {adaptee.specific_request()}", end="\n\n")

#     print("Client: But I can work with it via the Adapter:")
#     adapter = Adapter()
#     client_code(adapter)

In [ ]:
# dict-mapping vs match-case
# from timeit import timeit


# def perf_dict(inpt: str):
#     mapping = {
#         "foo": 1,
#         "bar": 2,
#         "baz": 3,
#         "qux": 4,
#         "quux": 5,
#     }
#     return mapping[inpt]

# def perf_matchcase(inpt: str):
#     match inpt:
#         case "foo":
#             return 1
#         case "bar":
#             return 2
#         case "baz":
#             return 3
#         case "qux":
#             return 4
#         case "quux":
#             return 5

# print(timeit('perf_dict("qux")', globals=globals(), number=10_000_000))
# print(timeit('perf_matchcase("qux")', globals=globals(), number=10_000_000))

In [ ]:
# wynnrepo init
# repo = WynnDataDatabase(config, logger)
# mockdata = MockWynnApi()
# p: PlayersResponse = mockdata.onlineuuids  # type: ignore
# ps: list[PlayerResponse] = mockdata.onlineplayerstats  # type: ignore
# gs: list[GuildResponse] = mockdata.onlineguildstats  # type: ignore

In [ ]:
# wynnrepo createtable
# db = WynnDataDatabase(config, None)
# await db.character_history_repository.create_table()
# await db.character_info_repository.create_table()
# await db.guild_history_repository.create_table()
# await db.guild_info_repository.create_table()
# await db.guild_member_history_repository.create_table()
# await db.kans_uptime_repository.create_table()
# await db.online_players_repository.create_table()
# await db.player_activity_history_repository.create_table()
# await db.player_history_repository.create_table()
# await db.player_info_repository.create_table()

In [ ]:
# wynnrepo insert
# print(await repo.character_history_repository.insert(CharacterHistory.from_responses(ps)))
# print(await repo.character_info_repository.insert(CharacterInfo.from_responses(ps)))
# print(await repo.guild_history_repository.insert(GuildHistory.from_responses(gs)))
# print(await repo.guild_info_repository.insert(GuildInfo.from_responses(gs)))
# print(await repo.guild_member_history_repository.insert(GuildMemberHistory.from_responses(gs)))
# print(await repo.online_players_repository.insert(OnlinePlayers.from_response(p)))
# # print(await repo.player_activity_history_repository.insert())
# print(await repo.player_history_repository.insert(PlayerHistory.from_responses(ps)))
# print(await repo.player_info_repository.insert(PlayerInfo.from_responses(ps)))

In [ ]:
# wynnrepo count
# print(await repo.character_history_repository.count())
# print(await repo.character_info_repository.count())
# print(await repo.guild_history_repository.count())
# print(await repo.guild_info_repository.count())
# print(await repo.guild_member_history_repository.count())
# print(await repo.online_players_repository.count())
# # print(await repo.player_activity_history_repository.count())
# print(await repo.player_history_repository.count())
# print(await repo.player_info_repository.count())

In [ ]:
# list of objects: `in`, `min()`, `max()`
# class Foo:
#     def __init__(self, weight: int, value: str):
#         self.weight = weight
#         self.value = value

#     def __eq__(self, value: object) -> bool:
#         return self.value == value

#     def __lt__(self, other: Foo) -> bool:
#         return self.weight < other.weight

# ch = 'abcdefghijklmnopqrstuvwxyz'
# foos = []
# for i in range(26):
#     foos.append(Foo(i, ch[i]))

# # 'value' in 'iterable' checks for equality of 'value' with each element in 'iterable'
# print('g' in foos)
# # min and max use the __lt__ magic method to compare elements in 'iterable'
# print(min(foos).value)
# print(max(foos).value)

In [ ]:
# dbmodel creation algorithm perfomance compariosn
# from tests.mock_wynnapi import MockWynnApi
# from vindicator.api.wynn.player_response import PlayerResponse
# from vindicator.db.wynndata.model.character_history.character_history import CharacterHistory
# from vindicator.db.wynndata.model.character_info.character_info import CharacterInfo
# from vindicator.db.wynndata.model.player_history.player_history import PlayerHistory
# from vindicator.db.wynndata.model.player_info.player_info import PlayerInfo

# mockwynnapi = MockWynnApi()
# data: list[PlayerResponse] = mockwynnapi.onlineplayerstats  # type: ignore

# def withloop():
#     playerinfo = []
#     characterinfo = []
#     playerhistory = []
#     characterhistory = []
#     for resp in data:
#         playerinfo.extend(PlayerInfo.from_responses((resp,)))
#         characterinfo.extend(CharacterInfo.from_responses((resp,)))
#         playerhistory.extend(PlayerHistory.from_responses((resp,)))
#         characterhistory.extend(CharacterHistory.from_responses((resp,)))

# def without():
#     playerinfo: tuple[PlayerInfo, ...] = PlayerInfo.from_responses(data)
#     characterinfo: tuple[CharacterInfo, ...] = CharacterInfo.from_responses(data)
#     playerhistory: tuple[PlayerHistory, ...] = PlayerHistory.from_responses(data)
#     characterhistory: tuple[CharacterHistory, ...] = CharacterHistory.from_responses(data)

# cProfile.run('withloop()', sort='time')
# cProfile.run('without()', sort='time')

In [ ]:
# += vs extend()
# r = range(50_000_000)
# def add():
#     l = []
#     l += tuple(r)
# def extend():
#     l = []
#     l.extend(r)

# cProfile.run('add()')
# cProfile.run('extend()')

In [ ]:
# moto-bot repository structure
# from __future__ import annotations
# ID = TypeVar('ID', contravariant=True)
# T = TypeVar('T')
# S = TypeVar('S')
# U = TypeVar('U', bound=type)

# def interface(t: U) -> U: return t
# def abstract(t: U) -> U: return t

# @interface
# class Table(Generic[T, ID], Protocol):
#     def create(self, entity: T) -> bool: ...
#     def exists(self, id_: ID) -> bool: ...
#     def count(self) -> float: ...
#     def findOne(self, id_: ID) -> None | T: ...
#     def findAll(self) -> None | list[T]: ...
#     def update(self, entity: T) -> bool: ...
#     def delete(self, id_: ID) -> bool: ...

# @abstract
# class MariaRepository(ABC, Generic[S]):
#     def __init__(self, db: Any, logger: Any) -> None:
#         self._db: Any = db
#         self._logger: Any = logger
#     def _prepareStatement(self, connection: Any, sql: str, objects: list[object]) -> Any: ...
#     def _execute(self, sql: str, strings: list[object]) -> bool: ...
#     # def _execute(self, connection: Any, sql: str, objects: list[object]) -> bool: ...
#     def _executeQuery(self, sql: str, objects: list[object]) -> None | Any: ...
#     def _logResponseException(self, e: Any) -> None: ...
#     def _bindAll(self, res: Any) -> list[S]: ...
#     @abstractmethod
#     def _bind(self, res: Any) -> S: ...

# @interface
# class GuildId(Protocol):
#     def getName(self) -> str: ...

# class Guild(GuildId):
#     def __init__(self, name: str, prefix: str, createdAt: dt) -> None:
#         self._name: str = name
#         self._prefix: str = prefix
#         self._createdAt: dt = createdAt
#     @override
#     def getName(self) -> str: return self._name
#     def getPrefix(self) -> str: return self._prefix
#     def getCreatedAt(self) -> dt: return self._createdAt

# @interface
# class CharacterHistoryBase(Table[Guild, GuildId], Protocol):
#     def findAllIn(self, guildNames: list[str]) -> None | list[Guild]: ...
#     def findAllCaseInsensitive(self, guildName: str) -> None | list[Guild]: ...
#     def findAllByPrefix(self, prefix: str) -> None | list[Guild]: ...
#     def findAllByPrefixCaseInsensitive(self, prefix: str) -> None | list[Guild]: ...

# class CharacterHistoryTable(MariaRepository[Guild], CharacterHistoryBase):
#     """class::MariaGuildRepository"""
#     _DB_FORMAT: Any
#     def __init__(self, db: Any, logger: Any) -> None:
#         super().__init__(db, logger)
#     @override
#     def _bind(self, res: Any) -> Guild: ...
#     @override
#     def create(self, entity: Guild) -> bool: ...
#     @override
#     def exists(self, id_: GuildId) -> bool: ...
#     @override
#     def count(self) -> float: ...
#     @override
#     def findOne(self, id_: GuildId) -> None | Guild: ...
#     @override
#     def findAllIn(self, guildNames: list[str]) -> None | list[Guild]: ...
#     def findAllCaseInsensitive(self, guildName: str) -> None | list[Guild]: ...
#     def findAllByPrefix(self, prefix: str) -> None | list[Guild]: ...
#     def findAllByPrefixCaseInsensitive(self, prefix: str) -> None | list[Guild]: ...
#     @override
#     def findAll(self) -> None | list[Guild]: ...
#     @override
#     def update(self, entity: Guild) -> bool: ...
#     @override
#     def delete(self, id_: GuildId) -> bool: ...
